In [1]:
import cv2
import numpy as np
from pathlib import Path

In [2]:
BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]

In [3]:
protoFile = "./openpose/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "./openpose/models/pose/mpi/pose_iter_160000.caffemodel"

In [4]:
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [5]:
image = cv2.imread("./all_angles/leftside.png")
image = cv2.resize(image, (512,512), interpolation=cv2.INTER_CUBIC)

output_img = np.zeros_like(image)

imageHeight, imageWidth, _ = image.shape

In [6]:
imgBlob = cv2.dnn.blobFromImage(image, 1.0/255, (imageHeight,imageWidth), (0,0,0), swapRB=False, crop=False)

In [7]:
# blob_reshaped = imgBlob.reshape(imgBlob.shape[2], imgBlob.shape[3], imgBlob.shape[1])

# blob_image = cv2.cvtColor(blob_reshaped, cv2.COLOR_BGR2RGB)

# cv2.imshow("blobs", blob_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
net.setInput(imgBlob)
output = net.forward()

In [9]:
Height = output.shape[2]
Width = output.shape[3]

In [10]:
print(Height, Width)

64 64


In [11]:
points = []

for i in range(len(BODY_PARTS)-1):
    probMap = output[0,i,:,:]
    
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    
    x = (imageWidth * point[0]) / Width
    y = (imageHeight * point[1]) / Height
    
    if prob > 0.1:
        cv2.circle(output_img, (int(x), int(y)), 5, (0,255,255), thickness = -1, lineType=cv2.FILLED)
        cv2.circle(image, (int(x), int(y)), 5, (0,255,255), thickness = -1, lineType=cv2.FILLED)
        cv2.putText(image, f"{i}", (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)
        
        points.append((int(x), int(y)))
    else:
        points.append(None)
        
cv2.imshow("output", output_img)
cv2.imshow("output2", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
imageCopy = output_img

for pair in POSE_PAIRS:
    partA = pair[0]          
    partA = BODY_PARTS[partA]
    partB = pair[1]
    partB = BODY_PARTS[partB]
    
    print(partA," 와 ", partB, " 연결\n")
    if points[partA] and points[partB]:
        cv2.line(imageCopy, points[partA], points[partB], (0, 255, 0), 2)


cv2.imshow("Output-Keypoints",imageCopy)
cv2.waitKey(0)
cv2.destroyAllWindows()

0  와  1  연결

1  와  2  연결

2  와  3  연결

3  와  4  연결

1  와  5  연결

5  와  6  연결

6  와  7  연결

1  와  14  연결

14  와  8  연결

8  와  9  연결

9  와  10  연결

14  와  11  연결

11  와  12  연결

12  와  13  연결

